In [ ]:
import pandas as pd
import numpy as np
import names
from IPython.display import display, HTML
from sqlalchemy import create_engine

display(HTML("<style>.container { width:100% !important; }</style>"))

# Creating a db

In [2]:
%load_ext sql

In [3]:
%sql sqlite:///library.db

'Connected: @library.db'

# Creating tables

In [80]:
%%sql
CREATE TABLE User(
    userID INTEGER PRIMARY KEY,
    firstName VARCHAR(30) NOT NULL,
    lastName VARCHAR(30) NOT NULL,
    age INTEGER NOT NULL,
    fines NUMERIC(5,2) DEFAULT 0,
    CHECK (age >= 7),
    CHECK (fines >= 0)
);

 * sqlite:///library.db
Done.


[]

In [81]:
%%sql
CREATE TABLE Librarian(
    librarianID INTEGER PRIMARY KEY,
    firstName VARCHAR(30) NOT NULL,
    lastName VARCHAR(30) NOT NULL,
    salary NUMERIC(6,2) DEFAULT 0,
    department VARCHAR(20) NOT NULL,
    CHECK (salary >= 0)
);

 * sqlite:///library.db
Done.


[]

In [82]:
%%sql
CREATE TABLE BorrowedItem(
    userID REFERENCES User(userID) NOT NULL,
    libraryItemID REFERENCES LibraryItem(libraryItemID) NOT NULL,
    dueDate DATETIME DEFAULT (date(CURRENT_TIMESTAMP, '+14 day')) NOT NULL,
    returnedDate DATETIME DEFAULT NULL,
    PRIMARY KEY (userID, libraryItemID, dueDate)
);

 * sqlite:///library.db
Done.


[]

In [83]:
%%sql
CREATE TABLE LibraryItem(
    libraryItemID INTEGER PRIMARY KEY,
    itemID REFERENCES Item(itemID) NOT NULL,
    toBeAdded BOOLEAN
);

 * sqlite:///library.db
Done.


[]

In [84]:
%%sql
CREATE TABLE Item(
    itemID INTEGER PRIMARY KEY,
    author VARCHAR(30),
    itemName VARCHAR(30) NOT NULL,
    type VARCHAR(15) NOT NULL
);

 * sqlite:///library.db
Done.


[]

In [85]:
%%sql
CREATE TABLE Event(
    startTS DATETIME NOT NULL,
    endTS DATETIME NOT NULL,
    room VARCHAR(10) NOT NULL,
    eventName VARCHAR(30) NOT NULL,
    audience VARCHAR(20),
    maxRegistrations INTEGER DEFAULT 0,
    artist VARCHAR(30),
    itemID REFERENCES Item(itemID),
    PRIMARY KEY (startTS, room),
    CHECK (maxRegistrations >= 0)
);

 * sqlite:///library.db
Done.


[]

In [86]:
%%sql
CREATE TABLE EventRegistration(
    startTS REFERENCES Event(startTS) NOT NULL,
    room REFERENCES Event(room) NOT NULL,
    userID REFERENCES User(userID) NOT NULL,
    PRIMARY KEY (startTS, room)
);

 * sqlite:///library.db
Done.


[]

# ASSERTIONS AND TRIGGERS

## List of Assertions

 - Maximum possible users <= some constant
 - Maximum employee salary <= budget

## List of Triggers

 - Users with fines cannot borrow items
 - Users cannot take items that are already checked out
 - Users cannot return items that have already been returned
 - Users cannot register for events at max capacity
 - Users cannot register multiple times for the same event
 - Users cannot register for past events

In [87]:
%%sql
CREATE TRIGGER prevent_multiple_returns
BEFORE UPDATE OF returnedDate ON BorrowedItem
FOR EACH ROW
WHEN NEW.returnedDate IS NOT NULL AND OLD.returnedDate IS NOT NULL
BEGIN
    SELECT RAISE(ABORT, 'The item has already been returned.');
END;

 * sqlite:///library.db
Done.


[]

In [88]:
%%sql
CREATE TRIGGER prevent_max_registrations
BEFORE INSERT ON EventRegistration
FOR EACH ROW
WHEN (
    SELECT COUNT(*) FROM EventRegistration WHERE startTS = NEW.startTS AND room = NEW.room
) >= (
    SELECT maxRegistrations FROM Event WHERE startTS = NEW.startTS AND room = NEW.room
)
BEGIN
    SELECT RAISE(ABORT, 'Event is at max capacity.');
END;

 * sqlite:///library.db
Done.


[]

In [89]:
%%sql
CREATE TRIGGER prevent_duplicate_event_registration
BEFORE INSERT ON EventRegistration
FOR EACH ROW
WHEN (
    SELECT COUNT(*) FROM EventRegistration
    WHERE startTS = NEW.startTS AND room = NEW.room AND userID = NEW.userID
) > 0
BEGIN
    SELECT RAISE(ABORT, 'The user has already registered for this event.');
END;

 * sqlite:///library.db
Done.


[]

In [5]:
%%sql
CREATE TRIGGER prevent_past_event_registration
BEFORE INSERT ON EventRegistration
FOR EACH ROW
WHEN (
    SELECT datetime(startTS) FROM Event WHERE startTS = NEW.startTS AND room = NEW.room
) < datetime('now')
BEGIN
    SELECT RAISE(ABORT, 'Cannot register for past events.');
END;

 * sqlite:///library.db
Done.


[]

In [7]:
%%sql
CREATE TRIGGER prevent_borrow_unavailable_item
BEFORE INSERT ON BorrowedItem
FOR EACH ROW
WHEN (
    SELECT COUNT(*) FROM BorrowedItem 
    WHERE libraryItemID = NEW.libraryItemID 
    AND returnedDate IS NULL
) > 0
BEGIN
    SELECT RAISE(ABORT, 'The item is not available for borrowing.');
END;

 * sqlite:///library.db
Done.


[]

In [11]:
%%sql
CREATE TRIGGER prevent_borrow_unavailable_item_tba
BEFORE INSERT ON BorrowedItem
FOR EACH ROW
WHEN (
    SELECT toBeAdded FROM LibraryItem WHERE libraryItemID = NEW.libraryItemID
) = 1
BEGIN
    SELECT RAISE(ABORT, 'The item is not available for borrowing.');
END;

 * sqlite:///library.db
Done.


[]

In [12]:
%%sql
SELECT * FROM sqlite_master WHERE type = 'trigger'; 

 * sqlite:///library.db
Done.


type,name,tbl_name,rootpage,sql
trigger,prevent_multiple_returns,BorrowedItem,0,"CREATE TRIGGER prevent_multiple_returnsBEFORE UPDATE OF returnedDate ON BorrowedItemFOR EACH ROWWHEN NEW.returnedDate IS NOT NULL AND OLD.returnedDate IS NOT NULLBEGIN SELECT RAISE(ABORT, 'The item has already been returned.');END"
trigger,prevent_max_registrations,EventRegistration,0,"CREATE TRIGGER prevent_max_registrationsBEFORE INSERT ON EventRegistrationFOR EACH ROWWHEN ( SELECT COUNT(*) FROM EventRegistration WHERE startTS = NEW.startTS AND room = NEW.room) >= ( SELECT maxRegistrations FROM Event WHERE startTS = NEW.startTS AND room = NEW.room)BEGIN SELECT RAISE(ABORT, 'Event is at max capacity.');END"
trigger,prevent_duplicate_event_registration,EventRegistration,0,"CREATE TRIGGER prevent_duplicate_event_registrationBEFORE INSERT ON EventRegistrationFOR EACH ROWWHEN ( SELECT COUNT(*) FROM EventRegistration WHERE startTS = NEW.startTS AND room = NEW.room AND userID = NEW.userID) > 0BEGIN SELECT RAISE(ABORT, 'The user has already registered for this event.');END"
trigger,prevent_past_event_registration,EventRegistration,0,"CREATE TRIGGER prevent_past_event_registrationBEFORE INSERT ON EventRegistrationFOR EACH ROWWHEN ( SELECT datetime(startTS) FROM Event WHERE startTS = NEW.startTS AND room = NEW.room) < datetime('now')BEGIN SELECT RAISE(ABORT, 'Cannot register for past events.');END"
trigger,prevent_borrow_unavailable_item,BorrowedItem,0,"CREATE TRIGGER prevent_borrow_unavailable_itemBEFORE INSERT ON BorrowedItemFOR EACH ROWWHEN ( SELECT COUNT(*) FROM BorrowedItem WHERE libraryItemID = NEW.libraryItemID AND returnedDate IS NULL) > 0BEGIN SELECT RAISE(ABORT, 'The item is not available for borrowing.');END"
trigger,prevent_borrow_unavailable_item_tba,BorrowedItem,0,"CREATE TRIGGER prevent_borrow_unavailable_item_tbaBEFORE INSERT ON BorrowedItemFOR EACH ROWWHEN ( SELECT toBeAdded FROM LibraryItem WHERE libraryItemID = NEW.libraryItemID) = 1BEGIN SELECT RAISE(ABORT, 'The item is not available for borrowing.');END"


# INDEXING

In [90]:
%%sql
SELECT * FROM sqlite_master WHERE type = 'index'; 

 * sqlite:///library.db
Done.


type,name,tbl_name,rootpage,sql
index,sqlite_autoindex_BorrowedItem_1,BorrowedItem,5,None
index,sqlite_autoindex_Event_1,Event,9,None
index,sqlite_autoindex_EventRegistration_1,EventRegistration,11,None


In [91]:
%%sql
--CREATE INDEX pk_user_id ON User(userID);
--CREATE INDEX pk_librarian_id ON Librarian(librarianID);
--CREATE INDEX pk_libraryitem_id ON LibraryItem(libraryItemID);
--CREATE INDEX pk_item_id ON Item(itemID);
CREATE INDEX ck_libraryitem_id ON LibraryItem(itemID);
CREATE INDEX ck_event_room ON Event(room);
CREATE INDEX ck_eventregistration_userid ON EventRegistration(userID);


 * sqlite:///library.db
Done.
Done.
Done.


[]

# Populating tables

## Table: Item

### Books

In [92]:
books = pd.read_csv('..//data//books2.csv').iloc[:10]
books = books[['author', 'title']]
books.rename(columns={'title':'itemName'}, inplace=True)
books.loc[:,'type'] = 'book'
books.drop_duplicates(subset=['itemName'], inplace=True)
books.head(5)

,author,itemName,type
0,Suzanne Collins,The Hunger Games,book
1,"J.K. Rowling, Mary GrandPré (Illustrator)",Harry Potter and the Order of the Phoenix,book
2,Harper Lee,To Kill a Mockingbird,book
3,"Jane Austen, Anna Quindlen (Introduction)",Pride and Prejudice,book
4,Stephenie Meyer,Twilight,book


### Movies

In [93]:
movies = pd.read_csv('..//data/movies.csv')[:5]
movies.rename(columns={'title':'itemName'}, inplace=True)
for row in range(0,len(movies)):
    movies.loc[row,'itemName'] = movies.iloc[row]['itemName'][:-7]
movies.loc[:, 'type'] = 'movie'
movies.loc[:, 'author'] = 'nan'
movies.drop(columns=['movieId', 'genres'], inplace=True)
movies.head(5)

,itemName,type,author
0,Toy Story,movie,nan
1,Jumanji,movie,nan
2,Grumpier Old Men,movie,nan
3,Waiting to Exhale,movie,nan
4,Father of the Bride Part II,movie,nan


### Songs

In [94]:
songs_full = pd.read_csv('..//data/songs.csv')
songs_full.rename(columns={'song':'itemName','artist':'author'}, inplace=True)
songs = pd.DataFrame()
for row in range(0,5):
    random_song = songs_full.sample()
    songs = pd.concat([songs,random_song])
songs.loc[:,'type'] = 'song'
songs.drop(columns=['link', 'text'], inplace=True)
songs.head(5)

,author,itemName,type
47010,Otis Redding,That's What My Heart Needs,song
43014,Marilyn Manson,Moonshine,song
14073,Nicki Minaj,Fly,song
12934,Michael Bolton,Lost In The City,song
14574,Oasis,Flashbax,song


### Papers

In [95]:
paper1 = pd.DataFrame(pd.Series({'author':'Abigail See', 'itemName':'A Discourse-Aware Attention Model for Abstractive Summarization of Long Documents', 'type':'paper'})).transpose()
paper2 = pd.DataFrame(pd.Series({'author':'Martin Ester', 'itemName':'A Density-Based Algorithm for Discovering Clusters in Large Spatial Databases with Noise ', 'type':'paper'})).transpose()
papers = pd.concat([paper1, paper2])
papers

,author,itemName,type
0,Abigail See,A Discourse-Aware Attention Model for Abstract...,paper
0,Martin Ester,A Density-Based Algorithm for Discovering Clus...,paper


In [96]:
Item = pd.concat([books, movies, songs, papers])
Item.reset_index(drop=True, inplace=True)
Item.index.names = ['itemID']

In [97]:
for row in range(0,len(Item)):
    itemName = Item.iloc[row]['author']
    comma_index = itemName.find(',')
    if (comma_index>1):
        author_solo = itemName[0:comma_index]
        Item.loc[row,'author'] = author_solo


In [98]:
Item

,author,itemName,type
itemID,,,
0,Suzanne Collins,The Hunger Games,book
1,J.K. Rowling,Harry Potter and the Order of the Phoenix,book
2,Harper Lee,To Kill a Mockingbird,book
3,Jane Austen,Pride and Prejudice,book
4,Stephenie Meyer,Twilight,book
5,Markus Zusak (Goodreads Author),The Book Thief,book
6,George Orwell,Animal Farm,book
7,C.S. Lewis,The Chronicles of Narnia,book
8,J.R.R. Tolkien,J.R.R. Tolkien 4-Book Boxed Set: The Hobbit an...,book


In [99]:
Item.to_csv('..//data//Item.csv')

## LibraryItem

In [100]:
num_copy = 8
LibraryItem = Item.copy()
for i in range(num_copy):
    item_copy = Item.sample()
    LibraryItem = pd.concat([LibraryItem, item_copy])


In [101]:
LibraryItem.reset_index(inplace=True)
LibraryItem.index.names = ['libraryItemID']
LibraryItem.drop(columns=['author','itemName','type'], inplace=True)
LibraryItem['toBeAdded'] = None

In [102]:
LibraryItem.loc[12,'toBeAdded'] = True

In [103]:
LibraryItem.to_csv('..//data//LibraryItem.csv')

## Librarian

In [104]:
numLibrarians = 4
departments = ['admin', 'catalog', 'circulation', 'infotech']
Librarian = pd.DataFrame()
for librarianID in range(1,numLibrarians+1):
    firstName = names.get_first_name()
    lastName = names.get_last_name()
    department = np.random.choice(departments)

    salary = np.random.randint(40000,90000)
    salary = round(salary, -3)
        
    librarian = pd.DataFrame(pd.Series({'firstName':firstName, 'lastName':lastName, 'department':department, 'salary':salary})).transpose()
    Librarian = pd.concat([Librarian, librarian])
    

librarian = pd.DataFrame(pd.Series({'firstName':'Joe', 'lastName':'Black', 'department':'volunteer', 'salary':0})).transpose()
Librarian = pd.concat([Librarian, librarian])
    
Librarian.dropna(inplace=True)


In [105]:
Librarian.reset_index(drop=True, inplace=True)
Librarian.index.names = ['librarianID']
Librarian

,firstName,lastName,department,salary
librarianID,,,,
0,Polly,Dunn,catalog,47000
1,Paul,Mahle,infotech,45000
2,Jose,Hoskins,circulation,77000
3,Edwin,Powell,circulation,71000
4,Joe,Black,volunteer,0


In [106]:
Librarian.to_csv('..//data//Librarian.csv')

## User

In [107]:
numUsers = 30
User = pd.DataFrame()
for librarianID in range(numUsers):
    firstName = names.get_first_name()
    lastName = names.get_last_name()
    age = np.random.randint(7,75)
        
    user = pd.DataFrame(pd.Series({'firstName':firstName, 'lastName':lastName, 'age':age, 'fines': 0.00})).transpose()
    User = pd.concat([User, user])
    

In [108]:
User.reset_index(drop=True, inplace=True)
User.index.names = ['userID']
User.loc[14,'fines'] = 3.00
User.loc[28,'fines'] = 3.00
User

,firstName,lastName,age,fines
userID,,,,
0,Kermit,Mcloughlin,32,0.0
1,Dannie,Horton,40,0.0
2,John,Monge,36,0.0
3,June,Niday,36,0.0
4,Roy,Shaffer,61,0.0
5,Jefferson,Moss,8,0.0
6,Lawrence,Owens,27,0.0
7,Randell,Baker,37,0.0
8,Deloise,Luten,53,0.0


In [109]:
User.to_csv('..//data//User.csv')

In [110]:
engine = create_engine('sqlite:///library.db', echo=False)
Item.reset_index().to_sql('Item', con=engine, if_exists='append', index=False)
LibraryItem.reset_index().to_sql('LibraryItem', con=engine, if_exists='append', index=False)
Librarian.reset_index().to_sql('Librarian', con=engine, if_exists='append', index=False)
User.to_sql('User', con=engine, if_exists='append', index=False)


30

In [114]:
%%sql
SELECT * FROM Item;

 * sqlite:///library.db
Done.


itemID,author,itemName,type
0,Suzanne Collins,The Hunger Games,book
1,J.K. Rowling,Harry Potter and the Order of the Phoenix,book
2,Harper Lee,To Kill a Mockingbird,book
3,Jane Austen,Pride and Prejudice,book
4,Stephenie Meyer,Twilight,book
5,Markus Zusak (Goodreads Author),The Book Thief,book
6,George Orwell,Animal Farm,book
7,C.S. Lewis,The Chronicles of Narnia,book
8,J.R.R. Tolkien,J.R.R. Tolkien 4-Book Boxed Set: The Hobbit and The Lord of the Rings,book
9,Margaret Mitchell,Gone with the Wind,book


## Table: User

In [138]:
%%sql
DROP TABLE mytable

 * sqlite:///library.db
Done.


[]

In [4]:
%%sql

DROP TABLE Item;
DROP TABLE User;
DROP TABLE Librarian;
DROP TABLE LibraryItem;
DROP TABLE BorrowedItem;
DROP TABLE Event;
DROP TABLE EventRegistration;

 * sqlite:///library.db
Done.
Done.
Done.
Done.
Done.
Done.
Done.
(sqlite3.OperationalError) no such index: ck_libraryitem_id
[SQL: DROP INDEX ck_libraryitem_id;]
(Background on this error at: https://sqlalche.me/e/14/e3q8)
